In [1]:
import cli_csvms.engine

2022-07-14 10:47:38,399: Database initiated


In [2]:
%%sql
-- Frutas
CREATE TABLE lista_frutas (
    nm_fruta TEXT ,
    tp_fruta TEXT )
;
INSERT INTO lista_frutas VALUES ('banana','doce');
INSERT INTO lista_frutas VALUES ('limão','amargo');
INSERT INTO lista_frutas VALUES ('mexerica','doce');
INSERT INTO lista_frutas VALUES ('maçã','doce');

-- Corrige o cadastro
UPDATE lista_frutas
   SET nm_fruta = 'bergamota'
     , tp_fruta = 'azedo'
 WHERE nm_fruta = 'mexerica';

-- Tipos
CREATE TABLE tipo_frutas (
    tp_fruta TEXT,
    vl_fruta FLOAT);

INSERT INTO tipo_frutas VALUES ('doce',1.5);
INSERT INTO tipo_frutas VALUES ('azedo',2.3);
INSERT INTO tipo_frutas VALUES ('amargo',2.0);

-- Atualiza o valor das frutas azedas
UPDATE tipo_frutas
   SET vl_fruta = 2.5
 WHERE tp_fruta = 'azedo';

-- Vendas
CREATE TABLE venda_frutas (
    cod_venda INTEGER,
    nm_fruta TEXT,
    qtd_venda INTEGER);
INSERT INTO venda_frutas VALUES (1,'banana',10);
INSERT INTO venda_frutas VALUES (2,'banana',12);
INSERT INTO venda_frutas VALUES (3,'limão',5);
INSERT INTO venda_frutas VALUES (4,'banana',8);
INSERT INTO venda_frutas VALUES (5,'limão',8);

-- Altera o tipo da fruta 'limão' para 'azedo'
UPDATE lista_frutas
   SET tp_fruta = 'azedo'
 WHERE nm_fruta = 'limão';

-- Remove a 'bergamota' da lista de frutas :(
DELETE FROM lista_frutas 
 WHERE nm_fruta = 'bergamota';

-- Altera a quantidade vendida da venda 3 para 20
UPDATE venda_frutas
   SET qtd_venda = 20
 WHERE cod_venda = 3;

-- Salva todas as alterações
COMMIT;


2022-07-14 10:47:38,558: creating new data dictionary in data/catalog.json
2022-07-14 10:47:38,580: Table lista_frutas was created
2022-07-14 10:47:38,591: Row ('banana', 'doce') inserted
2022-07-14 10:47:38,592: Rows inserted: 1
2022-07-14 10:47:38,595: Row ('limão', 'amargo') inserted
2022-07-14 10:47:38,596: Rows inserted: 1
2022-07-14 10:47:38,598: Row ('mexerica', 'doce') inserted
2022-07-14 10:47:38,599: Rows inserted: 1
2022-07-14 10:47:38,601: Row ('maçã', 'doce') inserted
2022-07-14 10:47:38,601: Rows inserted: 1
2022-07-14 10:47:38,604: Row 2 updated with values ('bergamota', 'azedo')
2022-07-14 10:47:38,605: Rows updated: 1
2022-07-14 10:47:38,623: Table tipo_frutas was created
2022-07-14 10:47:38,635: Row ('doce', 1.5) inserted
2022-07-14 10:47:38,636: Rows inserted: 1
2022-07-14 10:47:38,640: Row ('azedo', 2.3) inserted
2022-07-14 10:47:38,642: Rows inserted: 1
2022-07-14 10:47:38,644: Row ('amargo', 2.0) inserted
2022-07-14 10:47:38,645: Rows inserted: 1
2022-07-14 10:47:

# Motor de Consultas

As consultas a seguir tem seu resultado obtido a partir do banco de dados definido pelo arquivo [frutas.sql](../sql/frutas.sql)

A partir da última implementação.
Mostrar todos os registros da tabela `lista_frutas`

In [3]:
%%sql
SELECT * FROM lista_frutas

2022-07-14 10:47:38,827: TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



Recuperar o valor unitário de cada fruta

In [4]:
%%sql
SELECT l.nm_fruta AS fruta
     , t.vl_fruta AS valor
  FROM lista_frutas l
     , tipo_frutas t
 WHERE l.tp_fruta = t.tp_fruta

2022-07-14 10:47:38,951: TABLE: default.(((l×t)σ)π)
   +------+------+
   |fruta |valor |
   +------+------+
  0|banana|   1.5|
  1| limão|   2.5|
  2|  maçã|   1.5|
   +------+------+



Mostrar o tipo das frutas vendidas

In [5]:
%%sql
SELECT v.cod_venda
     , l.tp_fruta
  FROM venda_frutas v
  INNER JOIN lista_frutas l ON v.nm_fruta = l.nm_fruta

2022-07-14 10:47:39,050: TABLE: default.((v⋈l)π)
   +------------+-----------+
   |cod_venda   |tp_fruta   |
   +------------+-----------+
  0|           1|       doce|
  1|           2|       doce|
  2|           3|      azedo|
  3|           4|       doce|
  4|           5|      azedo|
   +------------+-----------+



Mostrar o valor total de cada venda

In [6]:
%%sql
SELECT v.cod_venda
     , l.nm_fruta
     , v.qtd_venda
     , t.vl_fruta
     , t.vl_fruta * v.qtd_venda as total
  FROM venda_frutas v
  INNER JOIN lista_frutas l ON v.nm_fruta = l.nm_fruta
  INNER JOIN tipo_frutas t ON l.tp_fruta = t.tp_fruta


2022-07-14 10:47:39,257: TABLE: default.((((v⋈l)⋈t)Π)π)
   +------------+-----------+------------+-----------+------+
   |cod_venda   |nm_fruta   |qtd_venda   |vl_fruta   |total |
   +------------+-----------+------------+-----------+------+
  0|           1|     banana|          10|        1.5|  15.0|
  1|           2|     banana|          12|        1.5|  18.0|
  2|           3|      limão|          20|        2.5|  50.0|
  3|           4|     banana|           8|        1.5|  12.0|
  4|           5|      limão|           8|        2.5|  20.0|
   +------------+-----------+------------+-----------+------+



Mostrar os tipos que não possuem frutas

In [7]:
%%sql
SELECT t.tp_fruta `tipos sem frutas`
  FROM lista_frutas l
 RIGHT JOIN tipo_frutas t ON t.tp_fruta = l.tp_fruta
 WHERE l.tp_fruta is null

2022-07-14 10:47:39,354: TABLE: default.(((l⟖t)σ)π)
   +-----------------+
   |tipos sem frutas |
   +-----------------+
  0|           amargo|
   +-----------------+



Mostrar as frutas com vendas inferiores a 20

In [8]:
%%sql
SELECT DISTINCT f.nm_fruta AS "Frutas não vendidas"
  FROM lista_frutas f
  LEFT JOIN (
SELECT v.nm_fruta
     , p.vl_fruta * v.qtd_venda as total
  FROM venda_frutas v
  INNER JOIN lista_frutas l ON v.nm_fruta = l.nm_fruta
  INNER JOIN tipo_frutas p ON l.tp_fruta = p.tp_fruta) c ON f.nm_fruta = c.nm_fruta
  WHERE c.total < 20 
     OR c.total IS NULL

2022-07-14 10:47:39,504: TABLE: default.(((f⟕c)σ)π)
   +--------------------+
   |Frutas não vendidas |
   +--------------------+
  0|              banana|
  1|                maçã|
   +--------------------+

